In [4]:
'''PACKAGE IMPORTS'''
#For data analysis
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import offsetbox
from matplotlib.gridspec import GridSpec
import numpy as np
import seaborn as sns
import scipy
import xarray as xr
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import statsmodels.formula.api as smf
import datetime as dt
from itertools import cycle

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from math import log10, floor, pow, e, sqrt

from functions import *

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
'''Import Data Links'''
import_path = "./data/original-data/"

In [ ]:
'''Colors'''
pal = sns.color_palette('GnBu', 5)
pal


In [ ]:
'''Functions'''
def jitter(values,j):
    return values + np.random.normal(j,0.1,values.shape)

## Import and Setup

In [ ]:
'''Import Data'''
### Import Snow Data
s2data = xr.open_dataset(import_path + '01_cleanedsnowdataS2.nc')
s2data_df = s2data.to_dataframe().reset_index(drop = False)
s2data_df = s2data_df.replace('NaN', np.nan)
s2data_df = s2data_df.replace('nan', np.nan)
s2data_df.time = pd.to_datetime(s2data_df.time)
s2data_df['watershed'] = 'S2'

s6data = xr.open_dataset(import_path + '01_cleanedsnowdataS6.nc')
s6data_df = s6data.to_dataframe().reset_index(drop = False)
s6data_df = s6data_df.replace('NaN', np.nan)
s6data_df = s6data_df.replace('nan', np.nan)
s6data_df.time = pd.to_datetime(s6data_df.time)
s6data_df['watershed'] = 'S6'

allSnow_df = pd.concat([s6data_df, s2data_df]).reset_index()

### Import LAI Data
s2LAI = pd.read_csv(import_path + "S2_winterLAI_calibrated.csv")
s6LAI = pd.read_csv(import_path + "S6_winterLAI_calibrated.csv")
lai_df = pd.concat([s2LAI, s6LAI]).reset_index()

### Import forestry Data -- summarized by stake
forestInv = pd.read_csv(import_path + 'VegData_LAI_Snow.csv')

### Import Weekly Met Data
weeklyMet = pd.read_csv(import_path + "WeeklyATMSummary.csv")

# Import SWE data
SWE = pd.read_csv(import_path + "2023SWE_data.csv", 
                parse_dates = ['DATE'])

In [ ]:
'''Add Snow Year'''
#Snow year
allSnow_df['SYear'] = [2024 if allSnow_df.time[i] > pd.to_datetime('09-01-2023') else 2023 for i in range(0, len(allSnow_df.time))]

#Date of snow year
allSnow_df['SDOY'] = [(allSnow_df.time[i] - pd.to_datetime('09-01-2023')).days if allSnow_df.SYear[i] == 2024 else (allSnow_df.time[i] - pd.to_datetime('09-01-2022')).days for i in range(0, len(allSnow_df.time))]
allSnow_df.head()

In [ ]:
#Retrofit one veg data outlier
#Stake S234 -- labelled all bog trees at dominant when they probably should have been codominant
forestInv.loc[18,  'Co'] = [20]
forestInv.loc[18,  'Dom'] = [0]
forestInv[forestInv.Stake_ID == 'S234']

In [ ]:
'''Organize all data into one data frame'''
#Remove rows on allSnow containing NaNs (these are grid locations where data was not taken)
allSnow_nan = allSnow_df.dropna(how = 'any')

#Take peak snow depth per site
peakSnow = allSnow_nan.groupby(['stakes', 'northing', 'easting', 'zones', 'aspect', 'slope', 'watershed', 'SYear'])['depths'].max().reset_index()
peakSnow = peakSnow.rename(columns = {'depths' : 'maxDepth'})

#Take average snow depth per site
avgSnow = allSnow_nan.groupby(['stakes', 'northing', 'easting', 'zones', 'aspect', 'slope', 'watershed', 'SYear'])['depths'].mean().reset_index()
avgSnow = avgSnow.rename(columns = {'depths' : 'meanDepth'})

#Add in other columns aggregated by year, joined on stakes
## LAI
lai_sub = lai_df[['Stake_ID', 'OLS Prediction Ring 5']]
peakData = pd.merge(peakSnow, lai_sub, left_on = 'stakes', right_on = 'Stake_ID')
peakData = peakData.drop(columns = 'Stake_ID')
peakData = peakData.rename(columns = {'OLS Prediction Ring 5' : 'lai5ring'})

avgData = pd.merge(avgSnow, lai_sub, left_on = 'stakes', right_on = 'Stake_ID')
avgData = avgData.drop(columns = 'Stake_ID')
avgData = avgData.rename(columns = {'OLS Prediction Ring 5' : 'lai5ring'})

## Veg Survey
veg_sub = forestInv[['Stake_ID', 'DIST_M', 'DBH_CM', 'DIST_M_SD', 'DBH_CM_SD', 'n', 'basalArea_m2', 'Co', 'Dom', 'Int', 'Sup', 'prop_Coniferous']]
peakData = pd.merge(peakData, veg_sub, left_on = 'stakes', right_on = 'Stake_ID')
peakData = peakData.drop(columns = 'Stake_ID')
peakData = peakData.rename(columns = {'DIST_M' : 'avgDist_m',
                                    'DBH_CM' : 'avgDBH_cm',
                                    'DIST_M_SD' : 'sdDist_m',
                                    'DBH_CM_SD' : 'sdDBH_cm',
                                    'n' : 'nTrees',
                                    'Co' : 'nCo',
                                    'Dom' : 'nDom',
                                    'Int' : 'nInt',
                                    'Sup' : 'nSup',
                                    'prop_Coniferous' : 'pCon'})

avgData = pd.merge(avgData, veg_sub, left_on = 'stakes', right_on = 'Stake_ID')
avgData = avgData.drop(columns = 'Stake_ID')
avgData = avgData.rename(columns = {'DIST_M' : 'avgDist_m',
                                    'DBH_CM' : 'avgDBH_cm',
                                    'DIST_M_SD' : 'sdDist_m',
                                    'DBH_CM_SD' : 'sdDBH_cm',
                                    'n' : 'nTrees',
                                    'Co' : 'nCo',
                                    'Dom' : 'nDom',
                                    'Int' : 'nInt',
                                    'Sup' : 'nSup',
                                    'prop_Coniferous' : 'pCon'})

In [ ]:
avgData.head()

## Melt Data -- need to fix

In [ ]:
'''Import Melt Data'''
melt = pd.read_csv(import_path + 'snowMeltRates.csv')
melt.head()

In [ ]:
#Merge Data with other descriptors for analysis
meltData = pd.merge(melt[['stakes', 'MaxDate_meas', 'firstSnow_meas', 'lastSnow_meas', 'accumulate_meas', 'melt_meas']],
                    peakData,
                    on = 'stakes')
meltData.columns

In [ ]:
#Remove 'days' text from columns
meltData['melt_meas'] = [int(a[0:-4]) for a in meltData.melt_meas]
meltData['accumulate_meas'] = [int(a[0:-4]) for a in meltData.accumulate_meas]

## General Snow Analysis

In [ ]:
allSnow_df.head()

In [ ]:
###Plot winter snow timeseries
fig = plt.figure(constrained_layout = True, 
    figsize = (9,6))

gs = GridSpec(2, 3, figure = fig)

ax = fig.add_subplot(gs[0, 0:2])
sns.lineplot(x = allSnow_df.SDOY, y = allSnow_df.depths, hue = allSnow_df.SYear, 
            palette = [pal[4], pal[2]], ax = ax)
ax.set_xlim(80, 250)
ax.set_ylim(0, 80)
ax.set_xlabel('Date of Snow Year')
ax.set_ylabel('Snow Depth, [cm]')

ax2 = fig.add_subplot(gs[0, 2])
sns.boxplot(data = allSnow_df, x = 'SYear', y = 'depths',
            showbox = False, showcaps = False, 
            medianprops={"color": "k", "linewidth": 2}, 
            palette = [pal[4], pal[2]],
            ax = ax2, zorder = 1)
sns.stripplot(data = allSnow_df, x = 'SYear', y = 'depths', dodge = True, 
              palette = [pal[4], pal[2]],
              ax = ax2, zorder = 0)
ax2.set_ylim(0, 80)
ax2.set_xlabel('Snow Year')
ax2.set_ylabel(' ')

In [ ]:
fig, [ax1, ax2] = plt.subplots(2, 1, figsize = (8, 8), 
                               sharex = True, 
                               sharey = True, 
                               layout = 'tight')

sns.lineplot(data = allSnow_df[allSnow_df.watershed == 'S2'], x = 'SDOY', y = 'depths', hue = 'SYear',
            units = 'stakes',
            estimator = None, 
            palette = [pal[4], pal[2]],
            ax = ax1)
ax1.set_xlim(80, 250)
ax1.set_ylim(0, 80)
ax1.set_ylabel('Snow Depth, [cm]')

sns.lineplot(data = allSnow_df[allSnow_df.watershed == 'S6'], x = 'SDOY', y = 'depths', hue = 'SYear',
            units = 'stakes',
            estimator = None, 
            palette = [pal[4], pal[2]],
            ax = ax2)
ax2.set_ylabel('Snow Depth, [cm]')
ax2.set_xlabel('Date of Snow Year')
plt.savefig('./figures/WaterSciCon/snowdepths.pdf')

## LAI Analysis

In [ ]:
fig = plt.figure(layout = "constrained", figsize = (9, 6))

gs = GridSpec(2, 3, figure = fig)
ax = fig.add_subplot(gs[0:2, 0:2])
sns.boxplot(data = peakData, x = 'zones', y = 'lai5ring',
    notch=False, showcaps=False,
    flierprops={"marker": "x"},
    boxprops={"facecolor": [0.4795847750865052, 0.7984621299500193, 0.7695501730103806, 0.3]},
    medianprops={"color": "red"},
    ax = ax)
ax.set_xlabel(' ')
ax.set_ylabel('5 Ring LAI')

ax2 = fig.add_subplot(gs[0, 2])
sns.regplot(data = peakData, x = 'lai5ring', y = 'maxDepth', color = pal[4])
results = smf.ols('maxDepth ~ lai5ring', data = peakData).fit()
ax2.text(1.7, 75, r'$ p $ = ' + str(round_sig(results.pvalues.lai5ring, 3)))
ax2.text(1.7, 72, r'$ R_{adj}^2 $ = ' + str(round_sig(results.rsquared_adj, 3)))

ax2.set_xlabel('5 Ring LAI')
ax2.set_ylabel('Peak Snow Depth [cm]')
ax2.set_xlim(min(peakData.lai5ring), max(peakData.lai5ring))

ax3 = fig.add_subplot(gs[1, 2])
sns.regplot(data = meltData, x = 'lai5ring', y = 'accumulate_meas', color = pal[4])
results2 = smf.ols('accumulate_meas ~ lai5ring', data = meltData).fit()
ax3.text(1.7, 115, r'$ p $ = ' + str(round_sig(results2.pvalues.lai5ring, 3)))
ax3.text(1.7, 109, r'$ R_{adj}^2 $ = ' + str(round_sig(results2.rsquared_adj, 3)))

ax3.set_xlabel('5 Ring LAI')
ax3.set_ylabel('Accumulation Season Length [days]')
ax3.set_xlim(min(meltData.lai5ring), max(meltData.lai5ring))

#plt.savefig(fig_savepath + 'laicorplot_boxplot.pdf', bbox_inches = 'tight')



In [ ]:
#Prep Data
bins = list(np.linspace(np.nanmin(peakData.pCon), np.nanmax(peakData.pCon), 4))
labels = ['Dense Deciduous', 'Mixed', 'Dense Coniferous']
peakData['Bin'] = pd.cut(peakData['pCon'], include_lowest = True, bins = bins, labels = labels)

#Plot
fig, [ax1, ax2, ax3] = plt.subplots(1, 3, figsize = (9, 3), 
                                    sharex = True, 
                                    sharey = True, 
                                    layout = 'tight')

sns.regplot(data = peakData[(peakData.Bin == 'Dense Coniferous') & (peakData.SYear == 2023)], x = 'lai5ring', y = 'maxDepth', 
            ax = ax1, 
            color = pal[4], 
            label = 'High Snow')
sns.regplot(data = peakData[(peakData.Bin == 'Dense Coniferous') & (peakData.SYear == 2024)], x = 'lai5ring', y = 'maxDepth', 
            ax = ax1, 
            color = pal[2], 
            label = 'Low Snow')
resultsDenseCon_high = smf.ols('maxDepth ~ lai5ring', data = peakData[(peakData.Bin == 'Dense Coniferous') & (peakData.SYear == 2023)]).fit()
resultsDenseCon_low = smf.ols('maxDepth ~ lai5ring', data = peakData[(peakData.Bin == 'Dense Coniferous') & (peakData.SYear == 2024)]).fit()
ax1.text(0.1, 92, r'$ p $ = ' + str(round_sig(resultsDenseCon_high.pvalues.lai5ring, 2)) + r', $ R_{adj}^2 $ = ' + str(round_sig(resultsDenseCon_high.rsquared_adj, 2)))
ax1.text(0.1, 84, r'$ p $ = ' + str(round_sig(resultsDenseCon_low.pvalues.lai5ring, 2)) + r', $ R_{adj}^2 $ = ' + str(round_sig(resultsDenseCon_low.rsquared_adj, 2)))
ax1.set_title('Coniferous (pCon > 0.66)', fontsize='small', loc='left')
ax1.set_ylabel('Peak Snow Depth [cm]')
ax1.set_xlabel('LAI')

sns.regplot(data = peakData[(peakData.Bin == 'Mixed') & (peakData.SYear == 2023)], x = 'lai5ring', y = 'maxDepth', 
            ax = ax2, 
            color = pal[4])
sns.regplot(data = peakData[(peakData.Bin == 'Mixed') & (peakData.SYear == 2024)], x = 'lai5ring', y = 'maxDepth', 
            ax = ax2, 
            color = pal[2])
resultsMixed_high = smf.ols('maxDepth ~ lai5ring', data = peakData[(peakData.Bin == 'Mixed') & (peakData.SYear == 2023)]).fit()
resultsMixed_low = smf.ols('maxDepth ~ lai5ring', data = peakData[(peakData.Bin == 'Mixed') & (peakData.SYear == 2024)]).fit()
ax2.text(0.1, 92, r'$ p $ = ' + str(round_sig(resultsMixed_high.pvalues.lai5ring, 2)) + r', $ R_{adj}^2 $ = ' + str(round_sig(resultsMixed_high.rsquared_adj, 2)))
ax2.text(0.1, 84, r'$ p $ = ' + str(round_sig(resultsMixed_low.pvalues.lai5ring, 2)) + r', $ R_{adj}^2 $ = ' + str(round_sig(resultsMixed_low.rsquared_adj, 2)))
ax2.set_title('Mixed (0.33 < pCon < 0.66)', fontsize='small', loc='left')
ax2.set_ylabel(' ')
ax2.set_xlabel('LAI')

sns.regplot(data = peakData[(peakData.Bin == 'Dense Deciduous') & (peakData.SYear == 2023)], x = 'lai5ring', y = 'maxDepth', 
            ax = ax3, 
            color = pal[4], 
            label = 'High Snow')
sns.regplot(data = peakData[(peakData.Bin == 'Dense Deciduous') & (peakData.SYear == 2024)], x = 'lai5ring', y = 'maxDepth', 
            ax = ax3, 
            color = pal[2], 
            label = 'Low Snow')
resultsDenseDec_high = smf.ols('maxDepth ~ lai5ring', data = peakData[(peakData.Bin == 'Dense Deciduous') & (peakData.SYear == 2023)]).fit()
resultsDenseDec_low = smf.ols('maxDepth ~ lai5ring', data = peakData[(peakData.Bin == 'Dense Deciduous') & (peakData.SYear == 2024)]).fit()
ax3.text(0.1, 92, r'$ p $ = ' + str(round_sig(resultsDenseDec_high.pvalues.lai5ring, 2)) + r', $ R_{adj}^2 $ = ' + str(round_sig(resultsDenseDec_high.rsquared_adj, 2)))
ax3.text(0.1, 84, r'$ p $ = ' + str(round_sig(resultsDenseDec_low.pvalues.lai5ring, 2)) + r', $ R_{adj}^2 $ = ' + str(round_sig(resultsDenseDec_low.rsquared_adj, 2)))
ax3.set_title('Deciduous (pCon < 0.33)', fontsize='small', loc='left')
ax3.set_ylabel(' ')
ax3.set_xlabel('LAI')

ax1.set_xlim(0, max(peakData.lai5ring))
ax1.set_ylim(10, 100)
ax3.legend(bbox_to_anchor = (1, 1))

#plt.savefig(fig_savepath + 'covertype_lairegression.pdf')

## Principle Component Analysis

In [ ]:
'''Set PCA Data'''
#Switch this depending on average/maximum snow depth considerations
pca_data = peakData

#remove strings
pca_data_forplot = pca_data.drop(columns = ['stakes', 'zones', 'watershed', 'Bin'])

#remove depths 
pca_data = pca_data_forplot.drop(columns = 'maxDepth')

In [ ]:
'''Principle Component Analysis to Determine Snow Controls'''
#Standardize Data
scaler = StandardScaler()
scaler.fit(pca_data)
pca_data_scaled = scaler.transform(pca_data)
#Convert back to dataframe
pca_scaled = pd.DataFrame(data = pca_data_scaled, 
                            columns = pca_data.columns)


In [ ]:
pca_scaled.head()

In [ ]:
#Run PCA
pca = PCA(n_components = 10) #adjustable
pca.fit_transform(pca_data_scaled)

#Extract variance values
prop_var = pca.explained_variance_ratio_
eigenvalues = pca.explained_variance_

In [ ]:
#Scree Plot
PCnumbers = np.arange(pca.n_components_) + 1

fig, [ax, ax2] = plt.subplots(1, 2, figsize = (6, 3), 
                              sharex = True, 
                              tight_layout = True)

ax.plot(PCnumbers, 
         eigenvalues, 
         'o-',
         color = pal[4])
ax.hlines(1, xmax = 0, xmin = 12, colors = 'silver', linestyles = '--', zorder = -1)
ax.set_ylabel('Eigenvalue')
ax.set_xlabel('Principle Component')

ax2.plot(PCnumbers, 
         np.cumsum(prop_var), 
         'o-', 
         color = pal[4])
ax2.hlines(0.90, xmax = 0, xmin = 12, colors = 'silver', linestyles = '--', zorder = -1)

ax2.set_ylabel('Proportion of Variance')
ax2.set_xlabel('Principle Component')

ax.set_xlim(0, 11)
#plt.savefig(fig_savepath + 'PCAeigenvalues.pdf', bbox_inches = 'tight')
plt.show()
#Using Kaiser's rule we keep all principle components with eigenvalues above 1 and rerun (here 5)

In [ ]:
#Rerun PCA
pca2 = PCA(n_components = 5) #adjustable
pc = pca2.fit_transform(pca_data_scaled)

pca_snow = pd.DataFrame(data = pc, columns = ['PC1', 'PC2', 'PC3', 'PC4', 'PC5'])
pca_snow.head()

In [ ]:
'''BiPlot'''
def biplot(ax, data, score, coef, ipca1, ipca2, labels=None, leg=None):
 
    xs = score[:, ipca1]
    ys = score[:, ipca2]
    n = coef.shape[0]
    scalex = 1.0/(xs.max() - xs.min())
    scaley = 1.0/(ys.max() - ys.min())

    sns.scatterplot(x = jitter(xs * scalex, 0.1), y = jitter(ys * scaley, 0.1), 
                hue = data.maxDepth, 
                ax = ax,
                palette = 'GnBu',
                legend = leg)
 
    for i in range(n):
        ax.arrow(0, 0, coef[i, ipca1], 
                  coef[i, ipca2], color = 'silver',
                  alpha = 0.5)
        ax.text(coef[i, ipca1]* 1.15, 
                 coef[i, ipca2] * 1.15, 
                 labels[i], 
                 color = 'silver', 
                 ha = 'center', 
                 va = 'center')
 
    ax.set_xlabel("PC{}".format(ipca1 + 1))
    ax.set_ylabel("PC{}".format(ipca2 + 1)) 

    ax.set_xlim(-0.7, 0.7)
    ax.set_ylim(-0.7, 0.7)

    #plt.title('Biplot of PCA')   

    #plt.show()

 
fig, axs = plt.subplots(2, 2, figsize = (6.5,5), 
                        tight_layout = True, 
                        sharey = True)
for i in range(0, 2):
    for j in range(0, 2):
        if((i == 0) & (j == 0)):
            biplot(axs[i, j], pca_data_forplot, pc, np.transpose(pca.components_), i+j+1, 0, list(pca_data.columns), True)
        else:
            biplot(axs[i, j], pca_data_forplot, pc, np.transpose(pca.components_), i+j+1, 0, list(pca_data.columns))

axs[0, 0].legend(title = 'Max SD [cm]', bbox_to_anchor = (-0.25, 1))
#plt.savefig(fig_savepath + 'PCAplots.pdf', bbox_inches = 'tight')
plt.show()

### Variable Importance Analysis

In [ ]:
#Sort out PCA coefficients
pca_coefs = pd.DataFrame(data = abs(np.transpose(pca2.components_)), columns = ['PC1', 'PC2', 'PC3', 'PC4', 'PC5'], index = pca_data.columns)
pca_coefs.head()

In [ ]:
#Variable Importance Plots
fig, ax = plt.subplots(1, 1, figsize = (6, 4))

pca_coefs.plot(kind = 'bar', stacked = True, ax = ax, color = pal)

ax.set_xticklabels(['Northing', 'Easting', 'Aspect', 'Slope', 'Snow Year', 'LAI', 'Mean Dist', 'Mean DBH', 'SD Dist', 'SD DBH', '# Trees', 'BA', '# Co', '# Dom', '# Int', '# Sup', '% Conif'])
ax.legend(title = 'Component', bbox_to_anchor = (1,1))
ax.set_ylabel('Cumulative PCA coefficient')
#plt.savefig(fig_savepath + 'PCAcoef.pdf', bbox_inches = 'tight')
plt.show()

In [ ]:
#Version weighted by PCA explained percentage
pca_coefs_weighted = pca_coefs
for i in range(0, 5):
    pca_coefs_weighted.iloc[:, i] = pca_coefs_weighted.iloc[:, i]*prop_var[i]

pca_coefs_weighted.head()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (6, 4))

pca_coefs_weighted.plot(kind = 'bar', stacked = True, ax = ax, color = pal)

ax.set_xticklabels(['Northing', 'Easting', 'Aspect', 'Slope', 'Snow Year', 'LAI', 'Mean Dist', 'Mean DBH', 'SD Dist', 'SD DBH', '# Trees', 'BA', '# Co', '# Dom', '# Int', '# Sup', '% Conif'])
ax.legend(title = 'Component', bbox_to_anchor = (1,1))
ax.set_ylabel('Cumulative PCA coefficient, weighted')
#plt.savefig(fig_savepath + 'PCAcoef-weighted.pdf', bbox_inches = 'tight')
plt.show()

## Random Forest Variable Importance

In [ ]:
def calculate_pvalues(df):
    dfcols = pd.DataFrame(columns=df.columns)
    pvalues = dfcols.transpose().join(dfcols, how='outer')
    for r in df.columns:
        for c in df.columns:
            tmp = df[df[r].notnull() & df[c].notnull()]
            pvalues[r][c] = round(stats.pearsonr(tmp[r], tmp[c])[1], 4)
    return pvalues

In [ ]:
#Variable intercorrelation
pvals = calculate_pvalues(pca_data) 
sigvals = np.where(pvals < 0.05, pvals, np.nan)

sns.heatmap(pca_data.corr(),
                xticklabels=pca_data.columns,
                yticklabels=pca_data.columns, 
            cmap = 'vlag', 
            mask = sigvals, 
            center = 0, 
            vmin = -1, vmax = 1)

### Variable Importance for Peak Snow Depth

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
import shap

In [ ]:
#Split data
X_train_2023, X_test_2023, y_train_2023, y_test_2023 = train_test_split(pca_data[pca_data.SYear == 2023],
                                                                        pca_data_forplot[pca_data_forplot.SYear == 2023].maxDepth,
                                                                        test_size = 0.5, random_state = 48492)
X_train_2024, X_test_2024, y_train_2024, y_test_2024 = train_test_split(pca_data[pca_data.SYear == 2024],
                                                                        pca_data_forplot[pca_data_forplot.SYear == 2024].maxDepth,
                                                                        test_size = 0.5, random_state = 48492)

#Fit random forest regression with 100 trees
rf2023 = RandomForestRegressor(n_estimators = 100)
rf2023.fit(X_train_2023, y_train_2023)
rf2024 = RandomForestRegressor(n_estimators = 100)
rf2024.fit(X_train_2024, y_train_2024)

In [ ]:
#Model assessment
preds2023 = rf2023.predict(X_test_2023)
preds2024 = rf2024.predict(X_test_2024)

#Score
print('2023 High Snow Model\n')
print(rf2023.score(X_train_2023, y_train_2023))
print(rf2023.score(X_test_2023, y_test_2023))

print('\n2024 Low Snow Model\n')
print(rf2024.score(X_train_2024, y_train_2024))
print(rf2024.score(X_test_2024, y_test_2024))

#Overfitting problem

In [ ]:
#Sort and plot feature importance
sorted_idx2023 = rf2023.feature_importances_.argsort()
sorted_idx2024 = rf2024.feature_importances_.argsort()

fig,[[ax, ax3],[ax2, ax4]] = plt.subplots(2, 2, figsize = (10, 5), 
                              sharex = True, 
                              tight_layout = True)

#2023
#Without permutation
ax.barh(pca_data.columns[sorted_idx2023], rf2023.feature_importances_[sorted_idx2023], 
        color = pal[4])
#ax.set_yticklabels(['Aspect', 'Northing', '# Sup', '# Int', 'Mean Dist', '# Trees', 'Easting', 'Slope', 'Mean DBH', '# Dom', 'LAI', 'BA', '% Con', '# Co'])

#With permutation
perm_importance2023 = permutation_importance(rf2023, X_test_2023, y_test_2023)

sorted_idx2023 = perm_importance2023.importances_mean.argsort()
ax2.barh(pca_data[pca_data.SYear == 2023].columns[sorted_idx2023], perm_importance2023.importances_mean[sorted_idx2023], 
         color = pal[4])
#ax2.set_yticklabels(['# Dom', '# Sup', '# Int', 'Easting', 'Aspect', 'Mean Dist', 'BA', '# Trees', 'Slope', 'Northing', 'Mean DBH', 'LAI', '% Con', '# Co'])

#2024
#Without permutation
ax3.barh(pca_data.columns[sorted_idx2024], rf2024.feature_importances_[sorted_idx2024], 
        color = pal[4])
#ax.set_yticklabels(['Aspect', 'Northing', '# Sup', '# Int', 'Mean Dist', '# Trees', 'Easting', 'Slope', 'Mean DBH', '# Dom', 'LAI', 'BA', '% Con', '# Co'])

#With permutation
perm_importance2024 = permutation_importance(rf2024, X_test_2024, y_test_2024)

sorted_idx2024 = perm_importance2024.importances_mean.argsort()
ax4.barh(pca_data[pca_data.SYear == 2024].columns[sorted_idx2024], perm_importance2024.importances_mean[sorted_idx2024], 
         color = pal[4])
#ax2.set_yticklabels(['# Dom', '# Sup', '# Int', 'Easting', 'Aspect', 'Mean Dist', 'BA', '# Trees', 'Slope', 'Northing', 'Mean DBH', 'LAI', '% Con', '# Co'])


ax2.set_xlabel("Random Forest Feature Importance")
ax4.set_xlabel("Random Forest Feature Importance")


#plt.savefig(fig_savepath + 'randomForest-importance.pdf', bbox_inches = 'tight')
plt.show()

In [ ]:
#SHAP values
explainer2023 = shap.TreeExplainer(rf2023)
shap_values2023 = explainer2023.shap_values(X_test_2023)
explainer2024 = shap.TreeExplainer(rf2024)
shap_values2024 = explainer2024.shap_values(X_test_2024)


fig, ax = plt.subplots(1, 1)
shap.summary_plot(shap_values2023, X_test_2023,
                cmap = 'GnBu', 
                plot_size = (5, 5), 
                color_bar_label = 'Attribute Value', 
                use_log_scale = True, 
                show = False)
#ax.set_yticklabels(['# Co', 'Mean DBH', 'pCon', r'/sigma DBH', '# Trees', 'LAI', '# Dom', r'\sigma Distance', 'Northing', '# Sup', 'Mean Distance', 'Easting', 'Slope', '# Int', 'Aspect', 'Basal Area', 'Year'])
ax.set_yticklabels(['Year', 'Basal Area', 'Aspect', '# Int', 'Slope', 'Easting', 'Mean Distance', '# Sup', 'Northing', r'$\sigma$ Distance', '# Dom', 'LAI', '# Trees', r'$\sigma$ DBH', 'pCon', 'Mean DBH', '# Co'])
plt.savefig('./figures/WaterSciCon/randomForest-SHAP-highsnow.pdf', bbox_inches = 'tight')
plt.savefig('./figures/WaterSciCon/randomForest-SHAP-highsnow.jpeg', dpi = 300, bbox_inches = 'tight')
plt.show()


fig, ax = plt.subplots(1, 1)
shap.summary_plot(shap_values2024, X_test_2024,
                cmap = 'GnBu', 
                plot_size = (5, 5), 
                color_bar_label = 'Attribute Value', 
                use_log_scale = True, 
                show = False)
#ax.set_yticklabels([r'$\sigma$ Distance', '# Co', '# Trees', 'pCon', '# Sup', 'Mean DBH', 'Slope', r'$\sigma$ DBH', 'Mean Distance', 'Aspect', 'Basal Area', 'LAI', '# Dom', 'Easting', 'Northing', '# Int', 'Year'])
ax.set_yticklabels(['Year', '# Int', 'Northing', 'Easting', '# Dom', 'LAI', 'Basal Area', 'Aspect', 'Mean Distance', r'$\sigma$ DBH', 'Slope', 'Mean DBH', '# Sup', 'pCon', '# Trees', '# Co', r'$\sigma$ Distance' ])
plt.savefig('./figures/WaterSciCon/randomForest-SHAP-lowsnow.pdf', bbox_inches = 'tight')
plt.savefig('./figures/WaterSciCon/randomForest-SHAP-lowsnow.jpeg', dpi = 300, bbox_inches = 'tight')
plt.show()

### Variable Importance Using Reduced Dimensions

In [ ]:
pca_data

In [ ]:
#Split data
X_train_2023, X_test_2023, y_train_2023, y_test_2023 = train_test_split(pca_data[pca_data.SYear == 2023],
                                                                        pca_data_forplot[pca_data_forplot.SYear == 2023].maxDepth,
                                                                        test_size = 0.5, random_state = 48492)
X_train_2024, X_test_2024, y_train_2024, y_test_2024 = train_test_split(pca_data[pca_data.SYear == 2024],
                                                                        pca_data_forplot[pca_data_forplot.SYear == 2024].maxDepth,
                                                                        test_size = 0.5, random_state = 48492)

#Fit random forest regression with 100 trees
rf2023 = RandomForestRegressor(n_estimators = 100)
rf2023.fit(X_train_2023, y_train_2023)
rf2024 = RandomForestRegressor(n_estimators = 100)
rf2024.fit(X_train_2024, y_train_2024)

In [ ]:
#Model assessment
preds2023 = rf2023.predict(X_test_2023)
preds2024 = rf2024.predict(X_test_2024)

#Score
print('2023 High Snow Model\n')
print(rf2023.score(X_train_2023, y_train_2023))
print(rf2023.score(X_test_2023, y_test_2023))

print('\n2024 Low Snow Model\n')
print(rf2024.score(X_train_2024, y_train_2024))
print(rf2024.score(X_test_2024, y_test_2024))

#Overfitting problem

In [ ]:
#SHAP values
explainer2023 = shap.TreeExplainer(rf2023)
shap_values2023 = explainer2023.shap_values(X_test_2023)
explainer2024 = shap.TreeExplainer(rf2024)
shap_values2024 = explainer2024.shap_values(X_test_2024)


fig, ax = plt.subplots(1, 1)
shap.summary_plot(shap_values2023, X_test_2023,
                cmap = 'GnBu', 
                plot_size = (5, 5), 
                color_bar_label = 'Attribute Value', 
                use_log_scale = True, 
                show = False)
#ax.set_yticklabels(['# Co', 'Mean DBH', 'pCon', r'/sigma DBH', '# Trees', 'LAI', '# Dom', r'\sigma Distance', 'Northing', '# Sup', 'Mean Distance', 'Easting', 'Slope', '# Int', 'Aspect', 'Basal Area', 'Year'])
ax.set_yticklabels(['Year', 'Basal Area', 'Aspect', '# Int', 'Slope', 'Easting', 'Mean Distance', '# Sup', 'Northing', r'$\sigma$ Distance', '# Dom', 'LAI', '# Trees', r'$\sigma$ DBH', 'pCon', 'Mean DBH', '# Co'])
plt.savefig('./figures/WaterSciCon/randomForest-SHAP-highsnow.pdf', bbox_inches = 'tight')
plt.savefig('./figures/WaterSciCon/randomForest-SHAP-highsnow.jpeg', dpi = 300, bbox_inches = 'tight')
plt.show()


fig, ax = plt.subplots(1, 1)
shap.summary_plot(shap_values2024, X_test_2024,
                cmap = 'GnBu', 
                plot_size = (5, 5), 
                color_bar_label = 'Attribute Value', 
                use_log_scale = True, 
                show = False)
#ax.set_yticklabels([r'$\sigma$ Distance', '# Co', '# Trees', 'pCon', '# Sup', 'Mean DBH', 'Slope', r'$\sigma$ DBH', 'Mean Distance', 'Aspect', 'Basal Area', 'LAI', '# Dom', 'Easting', 'Northing', '# Int', 'Year'])
ax.set_yticklabels(['Year', '# Int', 'Northing', 'Easting', '# Dom', 'LAI', 'Basal Area', 'Aspect', 'Mean Distance', r'$\sigma$ DBH', 'Slope', 'Mean DBH', '# Sup', 'pCon', '# Trees', '# Co', r'$\sigma$ Distance' ])
plt.savefig('./figures/WaterSciCon/randomForest-SHAP-lowsnow.pdf', bbox_inches = 'tight')
plt.savefig('./figures/WaterSciCon/randomForest-SHAP-lowsnow.jpeg', dpi = 300, bbox_inches = 'tight')
plt.show()

### Analysis by peatland zone

Bog

In [ ]:
pca_data_bog = peakData[peakData.zones == 'Bog']

#remove strings
pca_data_forplot_bog = pca_data_bog.drop(columns = ['stakes', 'zones', 'watershed'])

#remove depths 
pca_data_bog = pca_data_forplot_bog.drop(columns = 'maxDepth')


In [ ]:
#Split data
X_train_bog, X_test_bog, y_train_bog, y_test_bog = train_test_split(pca_data_bog, pca_data_forplot_bog.maxDepth, test_size = 0.5, random_state = 48492)

#Fit random forest regression with 100 trees
rf = RandomForestRegressor(n_estimators = 100)
rf.fit(X_train_bog, y_train_bog)

#SHAP values
explainer = shap.TreeExplainer(rf)
shap_values_bog = explainer.shap_values(X_test_bog)

shap.summary_plot(shap_values_bog, X_test_bog,
                cmap = 'GnBu', 
                plot_size = (5, 5), 
                color_bar_label = 'Snow Depth Value', 
                use_log_scale = True, 
                show = False)

plt.savefig(fig_savepath + 'randomForest-SHAP-bog.pdf', bbox_inches = 'tight')

Upland

In [ ]:
pca_data_upland = peakData[peakData.zones == 'Upland']

#remove strings
pca_data_forplot_upland = pca_data_upland.drop(columns = ['stakes', 'zones', 'watershed'])

#remove depths 
pca_data_upland = pca_data_forplot_upland.drop(columns = 'maxDepth')


In [ ]:
#Split data
X_train_upland, X_test_upland, y_train_upland, y_test_upland = train_test_split(pca_data_upland, pca_data_forplot_upland.maxDepth, test_size = 0.5, random_state = 48492)

#Fit random forest regression with 100 trees
rf = RandomForestRegressor(n_estimators = 100)
rf.fit(X_train_upland, y_train_upland)

#SHAP values
explainer = shap.TreeExplainer(rf)
shap_values_upland = explainer.shap_values(X_test_upland)

shap.summary_plot(shap_values_upland, X_test_upland,
                cmap = 'GnBu', 
                plot_size = (5, 5), 
                color_bar_label = 'Snow Depth Value', 
                use_log_scale = True, 
                show = False)

plt.savefig(fig_savepath + 'randomForest-SHAP-upland.pdf', bbox_inches = 'tight')

### Repeat Analysis for Snow Melt Rates

In [ ]:
#Split data
X_train, X_test, y_train, y_test = train_test_split(pca_data, meltData.melt_meas, test_size = 0.5, random_state = 48492)

#Fit random forest regression with 100 trees
rf = RandomForestRegressor(n_estimators = 100)
rf.fit(X_train, y_train)

In [ ]:
#Sort and plot feature importance
sorted_idx = rf.feature_importances_.argsort()

fig, [ax, ax2] = plt.subplots(2, 1, figsize = (6, 5), 
                              sharex = True, 
                              tight_layout = True)

#Without permutation
ax.barh(pca_data.columns[sorted_idx], rf.feature_importances_[sorted_idx], 
        color = pal[4])
#ax.set_yticklabels(['Aspect', 'Northing', '# Sup', '# Int', 'Mean Dist', '# Trees', 'Easting', 'Slope', 'Mean DBH', '# Dom', 'LAI', 'BA', '% Con', '# Co'])

#With permutation
perm_importance = permutation_importance(rf, X_test, y_test)

sorted_idx = perm_importance.importances_mean.argsort()
ax2.barh(pca_data.columns[sorted_idx], perm_importance.importances_mean[sorted_idx], 
         color = pal[4])
#ax2.set_yticklabels(['# Dom', '# Sup', '# Int', 'Easting', 'Aspect', 'Mean Dist', 'BA', '# Trees', 'Slope', 'Northing', 'Mean DBH', 'LAI', '% Con', '# Co'])

ax2.set_xlabel("Random Forest Feature Importance for Melt Season Length")

plt.savefig(fig_savepath + 'randomForest-importance-melt.pdf', bbox_inches = 'tight')
plt.show()

In [ ]:
#SHAP values
explainer = shap.TreeExplainer(rf)
shap_values = explainer.shap_values(X_test)

shap.summary_plot(shap_values, X_test,
                cmap = 'GnBu', 
                plot_size = (5, 5), 
                color_bar_label = 'Melt Rate', 
                use_log_scale = True, 
                show = False)

plt.savefig(fig_savepath + 'randomForest-SHAP-melt.pdf', bbox_inches = 'tight')

## Canopy Height Diversity

In [ ]:
#Shannon Diversity Index for Canopy Class
def calcH(co, dom, int, sup):
    tot = np.sum(co + dom + int + sup)

    #calc proportions
    pCo = co/tot
    pDom = dom/tot
    pInt = int/tot
    pSup = sup/tot

    #calc index masking any zeroes
    H = np.sum(pCo*np.log(pCo, where = (pCo != 0)) + 
               pDom*np.log(pDom, where = (pDom != 0)) +
               pInt*np.log(pInt, where = (pInt != 0)) + 
               pSup*np.log(pSup, where = (pSup != 0)))

    return -H

#Hill-Simpson Diversity Index for Canopy Class
def calcS(co, dom, int, sup):
    tot = np.sum(co + dom + int + sup)

    #calc proportions
    pCo = co/tot
    pDom = dom/tot
    pInt = int/tot
    pSup = sup/tot

    #calc index masking any zeroes
    S = np.sum(pow(pCo, 2) + pow(pDom, 2) + pow(pInt, 2) + pow(pSup, 2))

    return 1/S

#Proportion of co-dominant trees
def calcpCoDom(co, dom, int, sup):
    tot = np.sum(co + dom + int + sup)

    #calc proportions
    pCo = co/tot

    return pCo

dd = pca_data_forplot
dd['pCoDom'] = [calcpCoDom(dd.nCo[i], dd.nDom[i], dd.nInt[i], dd.nSup[i]) for i in dd.index]
dd['H'] = [calcH(dd.nCo[i], dd.nDom[i], dd.nInt[i], dd.nSup[i]) for i in dd.index]
dd['S'] = [calcS(dd.nCo[i], dd.nDom[i], dd.nInt[i], dd.nSup[i]) for i in dd.index]

In [ ]:
dd.head()

In [ ]:
fig, [ax2, ax3, ax4, ax] = plt.subplots(1, 4, figsize = (12, 3), 
                              sharey = True, 
                              tight_layout = True)

#Simpson Hill Diversity Index
dd['S2'] = [pow(a, 2) for a in dd.S]
sns.regplot(x = dd[(dd.SYear == 2023)].S2, y = dd[(dd.SYear == 2023)].maxDepth, order = 2, ax = ax2, color = pal[4])
sns.regplot(x = dd[(dd.SYear == 2024)].S2, y = dd[(dd.SYear == 2024)].maxDepth, order = 2, ax = ax2, color = pal[2])
#Do regression and add to plot
results2_high = smf.ols('maxDepth ~ S2', data = dd[(dd.SYear == 2023)]).fit()
results2_low = smf.ols('maxDepth ~ S2', data = dd[(dd.SYear == 2024)]).fit()
ax2.text(1.2, 92,
        r'$ p $ = ' + str(round_sig(results2_high.pvalues.S2, 2)) + r', $ R_{adj}^2 $ = ' + str(round_sig(results2_high.rsquared_adj, 2)), 
        color = pal[4])
ax2.text(1.2, 84, 
        r'$ p $ = ' + str(round_sig(results2_low.pvalues.S2, 2)) + r', $ R_{adj}^2 $ = ' + str(round_sig(results2_low.rsquared_adj, 2)), 
        color = pal[2])

ax2.set_xlabel('Hill-Simpson Diversity Index')
ax2.set_ylabel('Snow Depth [m]')
ax2.set_xlim(min(dd.S), max(dd.S))

#pCoDom
sns.regplot(x = dd[(dd.SYear == 2023)].pCoDom, y = dd[(dd.SYear == 2023)].maxDepth, order = 2, ax = ax3, color = pal[4])
sns.regplot(x = dd[(dd.SYear == 2024)].pCoDom, y = dd[(dd.SYear == 2024)].maxDepth, order = 2, ax = ax3, color = pal[2])
#Do regression and add to plot
dd['pCoDom2'] = [pow(a, 2) for a in dd.pCoDom]
results3_high = smf.ols('maxDepth ~ pCoDom2', data = dd[(dd.SYear == 2023)]).fit()
results3_low = smf.ols('maxDepth ~ pCoDom2', data = dd[(dd.SYear == 2024)]).fit()
ax3.text(0.1, 92, 
        r'$ p $ = ' + str(round_sig(results3_high.pvalues.pCoDom2, 2)) + r', $ R_{adj}^2 $ = ' + str(round_sig(results3_high.rsquared_adj, 2)), 
        color = pal[4])
ax3.text(0.1, 84,
        r'$ p $ = ' + str(round_sig(results3_low.pvalues.pCoDom2, 2)) + r', $ R_{adj}^2 $ = ' + str(round_sig(results3_low.rsquared_adj, 2)), 
        color = pal[2])

ax3.set_xlabel('Proportion of Co-Dominant Trees')
ax3.set_ylabel(' ')
ax3.set_xlim(min(dd.pCoDom), max(dd.pCoDom))

#pCo
sns.regplot(x = dd[(dd.SYear == 2023)].pCon, y = dd[(dd.SYear == 2023)].maxDepth, order = 1, ax = ax4, color = pal[4])
sns.regplot(x = dd[(dd.SYear == 2024)].pCon, y = dd[(dd.SYear == 2024)].maxDepth, order = 1, ax = ax4, color = pal[2])
#Do regression and add to plot
results4_high = smf.ols('maxDepth ~ pCon', data = dd[(dd.SYear == 2023)]).fit()
results4_low = smf.ols('maxDepth ~ pCon', data = dd[(dd.SYear == 2024)]).fit()
ax4.text(0.1, 92,
        r'$ p $ = ' + str(round_sig(results4_high.pvalues.pCon, 2)) + r', $ R_{adj}^2 $ = ' + str(round_sig(results4_high.rsquared_adj, 2)), 
        color = pal[4])
ax4.text(0.1, 84,
        r'$ p $ = ' + str(round_sig(results4_low.pvalues.pCon, 2)) + r', $ R_{adj}^2 $ = ' + str(round_sig(results4_low.rsquared_adj, 2)), 
        color = pal[2])

ax4.set_xlabel('Proportion of Coniferous Trees')
ax4.set_ylabel(' ')
ax4.set_xlim(min(dd.pCon), max(dd.pCon))

#sdDist_cm
sns.regplot(x = dd[(dd.SYear == 2023)].sdDist_m, y = dd[(dd.SYear == 2023)].maxDepth, order = 1, ax = ax, color = pal[4])
sns.regplot(x = dd[(dd.SYear == 2024)].sdDist_m, y = dd[(dd.SYear == 2024)].maxDepth, order = 1, ax = ax, color = pal[2])
#Do regression and add to plot
results_high = smf.ols('maxDepth ~ sdDist_m', data = dd[(dd.SYear == 2023)]).fit()
results_low = smf.ols('maxDepth ~ sdDist_m', data = dd[(dd.SYear == 2024)]).fit()
ax.text(1, 92,
        r'$ p $ = ' + str(round_sig(results_high.pvalues.sdDist_m, 2)) + r', $ R_{adj}^2 $ = ' + str(round_sig(results_high.rsquared_adj, 2)), 
        color = pal[4])
ax.text(1, 84,
        r'$ p $ = ' + str(round_sig(results_low.pvalues.sdDist_m, 2)) + r', $ R_{adj}^2 $ = ' + str(round_sig(results_low.rsquared_adj, 2)), 
        color = pal[2])

ax.set_xlabel('SD of Distance from Plot Center [m]')
ax.set_ylabel(' ')
ax.set_xlim(min(dd.sdDist_m), max(dd.sdDist_m))
ax.set_ylim(10, 100)

plt.savefig(fig_savepath + 'DiversityIndex-SnowDepth.pdf', bbox_inches = 'tight')

### Canopy Class Diversity Indicator #1

In [ ]:
def calcCCC(co, coConif, dom, domConif, int, intConif, sup, supConif):
    tot = np.sum(co + dom + int + sup)

    #calc proportions
    pCo = co/tot
    pDom = dom/tot
    pInt = int/tot
    pSup = sup/tot

    #calc coniferous proportions
    if(co > 0):
        pCoConif = coConif/co
    else:
        pCoConif = 0
    if(dom > 0):
        pDomConif = domConif/dom
    else:
        pDomConif = 0
    if(int > 0):
        pIntConif = intConif/int
    else:
        pIntConif = 0
    if(sup > 0):
        pSupConif = supConif/sup
    else:
        pSupConif = 0

    CCC = np.sum(pCo*np.log(pCo, where = (pCo != 0))*pCoConif + 
               pDom*np.log(pDom, where = (pDom != 0))*pDomConif +
               pInt*np.log(pInt, where = (pInt != 0))*pIntConif + 
               pSup*np.log(pSup, where = (pSup != 0))*pSupConif)


    return -CCC

In [ ]:
#Improved Shannon Diversity Metric -- Includes a proportion of each canopy class that is coniferous 

#Numerical Representation of Metric
Sup = []
Int = []
Co = []
Dom = []
SupCon = []
IntCon = []
CoCon = []
DomCon = []
H = []
S = []
CCC = []

#Generate fake data for a site
nTrees = 10

for nSup in np.arange(0, nTrees + 1):
    for nInt in np.arange(0, nTrees + 1 - nSup):
        for nCo in np.arange(0, nTrees + 1 - nSup - nInt):
            nDom = nTrees - nSup - nInt - nCo

            for SupConif in np.arange(0, nSup + 1):
                for IntConif in np.arange(0, nInt + 1):
                    for CoConif in np.arange(0, nCo + 1):
                        for DomConif in np.arange(0, nDom + 1):

                            #Save data
                            Sup.append(nSup)
                            Int.append(nInt)
                            Co.append(nCo)
                            Dom.append(nDom)

                            #Save data
                            SupCon.append(SupConif)
                            IntCon.append(IntConif)
                            CoCon.append(CoConif)
                            DomCon.append(DomConif)

                            #Calculate Shannon Diversity Index
                            H.append(calcH(nCo, nDom, nInt, nSup))
                            #Calculate Hill Simpson Diversity Index
                            S.append(calcS(nCo, nDom, nInt, nSup))
                            #Calculate Coniferous Based Canopy Diversity
                            CCC.append(calcCCC(nCo, CoConif, nDom, DomConif, nInt, IntConif, nSup, SupConif))                       

In [ ]:
#Append into dataframe
testData = pd.DataFrame({'Sup' : Sup, 
                         'Int' : Int, 
                         'Co' : Co, 
                         'Dom' : Dom,
                         'SupConif' : SupCon, 
                         'IntConif' : IntCon, 
                         'CoConif' : CoCon, 
                         'DomConif' : DomCon, 
                         'H' : H, 
                         'S' : S, 
                         'CCC' : CCC})

In [ ]:
fig = plt.figure(layout="constrained", figsize = (10, 4))

gs = GridSpec(1, 3, figure = fig)
ax = fig.add_subplot(gs[0, 0:2])
sns.scatterplot(data = testData, x = 'Co', y = 'H', ax = ax, label = 'Shannon', zorder = 5, color = pal[2])
sns.scatterplot(data = testData, x = 'Co', y = 'S', ax = ax, label = 'Simpson-Hill', color = pal[4])
sns.scatterplot(data = testData, x = 'Co', y = 'CCC', ax = ax, label = 'Canopy Class Conif (CCC)', color = 'green')
sns.lineplot(data = testData, x = 'Co', y = 'H', ax = ax, label = 'Shannon', zorder = 5, color = pal[2])
sns.lineplot(data = testData, x = 'Co', y = 'S', ax = ax, label = 'Simpson-Hill', color = pal[4])
sns.lineplot(data = testData, x = 'Co', y = 'CCC', ax = ax, label = 'Canopy Class Conif (CCC)', color = 'green')
ax.set_xlabel('Number of Co-dominant trees')
ax.set_ylabel('Diversity Index')

ax2 = fig.add_subplot(gs[0, 2])
sns.scatterplot(x = jitter(testData['Co'], 0), y = jitter(testData['CoConif'], 0), hue = testData['CCC'], palette = 'Greens', ax = ax2)
ax2.set_xlabel('Number of Co-dominant Trees')
ax2.set_ylabel('Number of Coniferous Co-dominant Trees')

plt.savefig(fig_savepath + 'diversityIndex_option1.pdf')

In [ ]:
#Complete with real data
#Data set-up

#Import
forestDataS2 = pd.read_excel(import_path + 'S2overstory_2023_compiled.xlsx', 
                             na_values = ['NAN'])
forestDataS6 = pd.read_excel(import_path + 'S6overstory_2023_compiled.xlsx', 
                             na_values = ['NAN'])
forestData = pd.concat([forestDataS2, forestDataS6])
forestData.head()

#Remove Nan rows and strip strings
forestData = forestData.dropna(how = 'any')
forestData['CC'] = [cl.strip() for cl in forestData.CC]

#Label coniferous and deciduous species
forestData['Conif'] = np.where(np.isin(forestData.SPECIES,
                ['Abies balsamea', 'Picea mariana', 'Larix laricina', 'Picea glauca', 'Pinus resinosa', 'Pinus Strobus']),
                'Coniferous', 'Deciduous')

#Sort into zones
forestData['zones'] = np.where(np.isin(forestData.SITE,
                ['S213', 'S221', 'S222', 'S223', 'S233', 'S234', 'S645', 'S634', 'S623', 'S612']), "Bog", 
                np.where(np.isin(forestData.SITE, ['S211', 'S212', 'S224', 'S225', 'S232', 'S235', 'S635', 'S624', 'S613', 'S611', 'S622', 'S633']),
                "Lagg", "Upland"))

#Group by site, canopy class, coniferous
forestData_group = forestData.groupby(['SITE', 'CC', 'Conif'])['SPECIES'].count().reset_index()

#Melt for data to be easier to manage
forestData_melt = forestData_group.pivot(index = 'SITE', columns = ['CC', 'Conif'], values = 'SPECIES')
forestData_melt = forestData_melt.fillna(0)

#Find sums and proportions
fs = forestData_melt
fs['nCo'] = fs['Co', 'Deciduous'] + fs['Co', 'Coniferous']
fs['nDom'] = fs['Dom', 'Deciduous'] + fs['Dom', 'Coniferous']
fs['nInt'] = fs['Int', 'Deciduous'] + fs['Int', 'Coniferous']
fs['nSup'] = fs['Sup', 'Deciduous'] + fs['Sup', 'Coniferous']

#Run diversity index algorithm
fs['CCC'] = [calcCCC(fs.nCo[i], fs['Co', 'Coniferous'][i],
                    fs.nDom[i], fs['Dom', 'Coniferous'][i],
                    fs.nInt[i], fs['Int', 'Coniferous'][i],
                    fs.nSup[i], fs['Sup', 'Coniferous'][i]) for i in range(0, len(fs.nCo))]

fs = fs.reset_index()

fs.head()

In [ ]:
#Subset and merge with snow data
diversityOne = pd.merge(peakSnow[['stakes', 'maxDepth', 'watershed']], fs[['CCC', 'SITE']], left_on = 'stakes', right_on = 'SITE')
diversityOne = diversityOne.rename(columns = {diversityOne.columns[3] : 'CCC',
                                            diversityOne.columns[4] : 'SITE'})
diversityOne.head()

In [ ]:
#Real data and Indicator 1 plot
fig, ax = plt.subplots(1, 1, figsize = (4, 4))

sns.regplot(data = diversityOne[diversityOne.watershed == 'S2'], x = 'CCC', y = 'maxDepth',
            ax = ax, color = pal[4], 
            label = 'S2')
sns.regplot(data = diversityOne[diversityOne.watershed == 'S6'], x = 'CCC', y = 'maxDepth',
            ax = ax, color = pal[2], 
            label = 'S6')

#Do regression and add to plot
resultsS2 = smf.ols('maxDepth ~ CCC', data = diversityOne[diversityOne.watershed == 'S2']).fit()
ax.text(0.05, 75, r'$ p_{S2} $ = ' + str(round_sig(resultsS2.pvalues.CCC, 3)))
ax.text(0.05, 72, r'$ R_{adj,S2}^2 $ = ' + str(round_sig(resultsS2.rsquared_adj, 3)))

resultsS6 = smf.ols('maxDepth ~ CCC', data = diversityOne[diversityOne.watershed == 'S6']).fit()
ax.text(0.05, 69, r'$ p_{S6} $ = ' + str(round_sig(resultsS6.pvalues.CCC, 3)))
ax.text(0.05, 66, r'$ R_{adj,S6}^2 $ = ' + str(round_sig(resultsS6.rsquared_adj, 3)))

ax.set_xlabel('Coniferous Canopy Class Diversity Index')
ax.set_ylabel('Snow Depth [cm]')
ax.set_xlim(np.nanmin(diversityOne.CCC), np.nanmax(diversityOne.CCC))

ax.legend()

plt.savefig(fig_savepath + 'DiversityIndex1-SnowDepth.pdf', bbox_inches = 'tight')

### Canopy Class Diversity Indicator #2

In [ ]:
def calcCCCa(co, coConif, dom, domConif, int, intConif, sup, supConif):
    tot = np.sum(co + dom + int + sup)

    #calc proportions
    pCo = co/tot
    pDom = dom/tot
    pInt = int/tot
    pSup = sup/tot

    #calc coniferout proportions
    if(co > 0):
        pCoConif = coConif/co
    else:
        pCoConif = 0
    if(dom > 0):
        pDomConif = domConif/dom
    else:
        pDomConif = 0
    if(int > 0):
        pIntConif = intConif/int
    else:
        pIntConif = 0
    if(sup > 0):
        pSupConif = supConif/sup
    else:
        pSupConif = 0
    

    CCCa = (pCo*np.log(pCo, where = (pCo != 0))+pCoConif*np.log(pCoConif,
                where = (pCoConif != 0))) + pDom*np.log(pDom, where = (pDom != 0))+pDomConif*np.log(pDomConif,
                where = (pDomConif != 0)) + pInt*np.log(pInt, where = (pInt != 0))+pIntConif*np.log(pIntConif,
                where = (pIntConif != 0)) + pSup*np.log(pSup, where = (pSup != 0))+pSupConif*np.log(pSupConif,
                where = (pSupConif != 0))

    return -CCCa

In [ ]:
#Improved Shannon Diversity Metric -- Includes a proportion of each canopy class that is coniferous 

#Numerical Representation of Metric
Sup = []
Int = []
Co = []
Dom = []
SupCon = []
IntCon = []
CoCon = []
DomCon = []
H = []
S = []
CCCa = []

#Generate fake data for a site
nTrees = 10

for nSup in np.arange(0, nTrees + 1):
    for nInt in np.arange(0, nTrees + 1 - nSup):
        for nCo in np.arange(0, nTrees + 1 - nSup - nInt):
            nDom = nTrees - nSup - nInt - nCo

            for SupConif in np.arange(0, nSup + 1):
                for IntConif in np.arange(0, nInt + 1):
                    for CoConif in np.arange(0, nCo + 1):
                        for DomConif in np.arange(0, nDom + 1):

                            #Save data
                            Sup.append(nSup)
                            Int.append(nInt)
                            Co.append(nCo)
                            Dom.append(nDom)

                            #Save data
                            SupCon.append(SupConif)
                            IntCon.append(IntConif)
                            CoCon.append(CoConif)
                            DomCon.append(DomConif)

                            #Calculate Shannon Diversity Index
                            H.append(calcH(nCo, nDom, nInt, nSup))
                            #Calculate Hill Simpson Diversity Index
                            S.append(calcS(nCo, nDom, nInt, nSup))
                            #Calculate Coniferous Based Canopy Diversity
                            CCCa.append(calcCCCa(nCo, CoConif, nDom, DomConif, nInt, IntConif, nSup, SupConif))                       

In [ ]:
#Append into dataframe
testData_adapted = pd.DataFrame({'Sup' : Sup, 
                         'Int' : Int, 
                         'Co' : Co, 
                         'Dom' : Dom,
                         'SupConif' : SupCon, 
                         'IntConif' : IntCon, 
                         'CoConif' : CoCon, 
                         'DomConif' : DomCon, 
                         'H' : H, 
                         'S' : S, 
                         'CCCa' : CCCa})

In [ ]:
fig = plt.figure(layout="constrained", figsize = (10, 4))

gs = GridSpec(1, 3, figure = fig)
ax = fig.add_subplot(gs[0, 0:2])
sns.scatterplot(data = testData_adapted, x = 'Co', y = 'H', ax = ax, label = 'Shannon', zorder = 5, color = pal[4])
sns.scatterplot(data = testData_adapted, x = 'Co', y = 'S', ax = ax, label = 'Simpson-Hill', color = pal[2])
sns.scatterplot(data = testData_adapted, x = 'Co', y = 'CCCa', ax = ax, label = 'Canopy Class Conif Adapted (CCCa)', color = 'green')
sns.lineplot(data = testData_adapted, x = 'Co', y = 'H', ax = ax, zorder = 5, color = pal[4])
sns.lineplot(data = testData_adapted, x = 'Co', y = 'S', ax = ax, color = pal[2])
sns.lineplot(data = testData_adapted, x = 'Co', y = 'CCCa', ax = ax, color = 'green')
ax.set_xlabel('Number of Co-dominant trees')
ax.set_ylabel('Diversity Index')

ax2 = fig.add_subplot(gs[0, 2])
sns.scatterplot(x = jitter(testData['Co'], 0), y = jitter(testData_adapted['CoConif'], 0), hue = testData_adapted['CCCa'], palette = 'Greens', ax = ax2)
ax2.set_xlabel('Number of Co-dominant Trees')
ax2.set_ylabel('Number of Coniferous Co-dominant Trees')

plt.savefig(fig_savepath + 'diversityIndex_option2.pdf')

In [ ]:
#Run diversity index algorithm on real data
fs['CCCa'] = [calcCCCa(fs.nCo[i], fs['Co', 'Coniferous'][i],
                    fs.nDom[i], fs['Dom', 'Coniferous'][i],
                    fs.nInt[i], fs['Int', 'Coniferous'][i],
                    fs.nSup[i], fs['Sup', 'Coniferous'][i]) for i in range(0, len(fs.nCo))]

fs.head()

In [ ]:
#Subset and merge with snow data
diversityTwo = pd.merge(peakSnow[['stakes', 'maxDepth', 'watershed']], fs[['CCCa', 'SITE']], left_on = 'stakes', right_on = 'SITE')
diversityTwo = diversityTwo.rename(columns = {diversityTwo.columns[3] : 'CCCa',
                                            diversityTwo.columns[4] : 'SITE'})
diversityTwo.head()

In [ ]:
#Real data and Indicator 1 plot
fig, ax = plt.subplots(1, 1, figsize = (4, 4))

sns.regplot(data = diversityTwo[diversityTwo.watershed == 'S2'], x = 'CCCa', y = 'maxDepth',
            ax = ax, color = pal[4], 
            label = 'S2')
sns.regplot(data = diversityTwo[diversityTwo.watershed == 'S6'], x = 'CCCa', y = 'maxDepth',
            ax = ax, color = pal[2], 
            label = 'S6')

#Do regression and add to plot
resultsS2 = smf.ols('maxDepth ~ CCCa', data = diversityTwo[diversityTwo.watershed == 'S2']).fit()
ax.text(0.05, 75, r'$ p_{S2} $ = ' + str(round_sig(resultsS2.pvalues.CCCa, 3)))
ax.text(0.05, 72, r'$ R_{adj,S2}^2 $ = ' + str(round_sig(resultsS2.rsquared_adj, 3)))

resultsS6 = smf.ols('maxDepth ~ CCCa', data = diversityTwo[diversityTwo.watershed == 'S6']).fit()
ax.text(0.05, 69, r'$ p_{S6} $ = ' + str(round_sig(resultsS6.pvalues.CCCa, 3)))
ax.text(0.05, 66, r'$ R_{adj,S6}^2 $ = ' + str(round_sig(resultsS6.rsquared_adj, 3)))

ax.set_xlabel('Coniferous Canopy Class Diversity Index')
ax.set_ylabel('Snow Depth [cm]')
ax.set_xlim(np.nanmin(diversityTwo.CCCa), np.nanmax(diversityTwo.CCCa))

ax.legend()

plt.savefig(fig_savepath + 'DiversityIndex2-SnowDepth.pdf', bbox_inches = 'tight')

In [ ]:
'''Data Setup but for Uplands'''
peakSnow_up = peakSnow[peakSnow.zones == 'Upland']

#select upland data
forestData_up = forestData[forestData.zones == 'Upland']

#Group by site, canopy class, coniferous
forestData_up_group = forestData_up.groupby(['SITE', 'CC', 'Conif'])['SPECIES'].count().reset_index()

#Melt for data to be easier to manage
forestData_up_melt = forestData_up_group.pivot(index = 'SITE', columns = ['CC', 'Conif'], values = 'SPECIES')
forestData_up_melt = forestData_up_melt.fillna(0)

#Find sums and proportions
fs_up = forestData_up_melt
fs_up['nCo'] = fs_up['Co', 'Deciduous'] + fs_up['Co', 'Coniferous']
fs_up['nDom'] = fs_up['Dom', 'Deciduous'] + fs_up['Dom', 'Coniferous']
fs_up['nInt'] = fs_up['Int', 'Deciduous'] + fs_up['Int', 'Coniferous']
fs_up['nSup'] = fs_up['Sup', 'Deciduous'] + fs_up['Sup', 'Coniferous']

#Run diversity index algorithm
fs_up['CCC'] = [calcCCC(fs_up.nCo[i], fs_up['Co', 'Coniferous'][i],
                    fs_up.nDom[i], fs_up['Dom', 'Coniferous'][i],
                    fs_up.nInt[i], fs_up['Int', 'Coniferous'][i],
                    fs_up.nSup[i], fs_up['Sup', 'Coniferous'][i]) for i in range(0, len(fs_up.nCo))]

fs_up = fs_up.reset_index()

fs_up.head()

In [ ]:
#Run diversity index algorithm on real data
fs_up['CCCa'] = [calcCCCa(fs_up.nCo[i], fs_up['Co', 'Coniferous'][i],
                    fs_up.nDom[i], fs_up['Dom', 'Coniferous'][i],
                    fs_up.nInt[i], fs_up['Int', 'Coniferous'][i],
                    fs_up.nSup[i], fs_up['Sup', 'Coniferous'][i]) for i in range(0, len(fs_up.nCo))]

fs_up.head()

In [ ]:
#Subset and merge with snow data
diversityTwo_up = pd.merge(peakSnow_up[['stakes', 'maxDepth', 'watershed']], fs_up[['CCCa', 'SITE']], left_on = 'stakes', right_on = 'SITE', how = 'inner')
diversityTwo_up = diversityTwo_up.rename(columns = {diversityTwo_up.columns[3] : 'CCCa',
                                            diversityTwo_up.columns[4] : 'SITE'})
diversityTwo_up.head()

In [ ]:
#Real data and Indicator 1 plot
fig, ax = plt.subplots(1, 1, figsize = (4, 4))

sns.regplot(data = diversityTwo_up[diversityTwo_up.watershed == 'S2'], x = 'CCCa', y = 'maxDepth',
            ax = ax, color = pal[4], 
            label = 'S2')
sns.regplot(data = diversityTwo_up[diversityTwo_up.watershed == 'S6'], x = 'CCCa', y = 'maxDepth',
            ax = ax, color = pal[2], 
            label = 'S6')

#Do regression and add to plot
resultsS2 = smf.ols('maxDepth ~ CCCa', data = diversityTwo_up[diversityTwo_up.watershed == 'S2']).fit()
ax.text(0.05, 75, r'$ p_{S2} $ = ' + str(round_sig(resultsS2.pvalues.CCCa, 3)))
ax.text(0.05, 72, r'$ R_{adj,S2}^2 $ = ' + str(round_sig(resultsS2.rsquared_adj, 3)))

resultsS6 = smf.ols('maxDepth ~ CCCa', data = diversityTwo_up[diversityTwo_up.watershed == 'S6']).fit()
ax.text(0.05, 69, r'$ p_{S6} $ = ' + str(round_sig(resultsS6.pvalues.CCCa, 3)))
ax.text(0.05, 66, r'$ R_{adj,S6}^2 $ = ' + str(round_sig(resultsS6.rsquared_adj, 3)))

ax.set_xlabel('Coniferous Canopy Class Diversity Index')
ax.set_ylabel('Snow Depth [cm]')
ax.set_xlim(np.nanmin(diversityTwo_up.CCCa), np.nanmax(diversityTwo_up.CCCa))

ax.legend()

plt.savefig(fig_savepath + 'DiversityIndex2-SnowDepth-uplands.pdf', bbox_inches = 'tight')

### Canopy Class Indicator #3

All Data

In [ ]:
#Calculate the proportion of coniferous codominant canopy
fs['CCCon'] = fs['Co', 'Coniferous']/fs['nCo']
fs['pCo'] = fs['nCo']/(fs['nCo'] + fs['nDom'] + fs['nInt'] + fs['nSup'])
fs['CCCon_ntrees'] = fs['Co', 'Coniferous']/(fs['nCo'] + fs['nDom'] + fs['nInt'] + fs['nSup'])
fs.head()

In [ ]:
#Subset and merge with snow data
diversityThree = pd.merge(peakData[['stakes', 'maxDepth', 'zones', 'watershed', 'aspect', 'slope', 'lai5ring', 'nTrees', 'pCon', 'SYear']], fs[['CCCon', 'pCo', 'SITE']], left_on = 'stakes', right_on = 'SITE', how = 'inner')
diversityThree = diversityThree.rename(columns = {diversityThree.columns[-3] : 'CCCon',
                                            diversityThree.columns[-2] : 'pCo', 
                                            diversityThree.columns[-1] : 'SITE'})
diversityThree.head()

In [ ]:
#calculate non-dimensional variable
diversityThree['PiInd'] = diversityThree.pCon / (diversityThree.lai5ring*diversityThree.pCo*diversityThree.CCCon)
diversityThree['logPiInd'] = np.log(diversityThree['PiInd'])

#Change infs to nans
diversityThree = diversityThree.replace([np.inf, -np.inf], np.nan)
diversityThree.head()

In [ ]:
#Real data and Indicator 1 plot
fig, ax = plt.subplots(1, 1, figsize = (4, 4))

sns.regplot(data = diversityThree[diversityThree.SYear == 2023], x = 'PiInd', y = 'maxDepth',
            logx = True,
            ax = ax, color = pal[4])
sns.regplot(data = diversityThree[diversityThree.SYear == 2024], x = 'PiInd', y = 'maxDepth',
            logx = True,
            ax = ax, color = pal[2])

#Do regression and add to plot
res1 = smf.ols('maxDepth ~ logPiInd', data = diversityThree[(diversityThree.SYear == 2023)]).fit()
res2 = smf.ols('maxDepth ~ logPiInd', data = diversityThree[(diversityThree.SYear == 2024)]).fit()
ax.text(5, 92,
        r'$ p $ = ' + str(round_sig(res1.pvalues.logPiInd, 2)) + r', $ R_{adj}^2 $ = ' + str(round_sig(res1.rsquared_adj, 2)), 
        color = pal[4])
ax.text(5, 84, r'$ p $ = ' + str(round_sig(res2.pvalues.logPiInd, 2)) + r', $ R_{adj}^2 $ = ' + str(round_sig(res2.rsquared_adj, 2)), 
        color = pal[2])

ax.set_xlabel('Coniferous Canopy Class Index')
ax.set_ylabel('Snow Depth [cm]')
ax.set_xlim(np.nanmin(diversityThree.PiInd), np.nanmax(diversityThree.PiInd))
ax.set_ylim(10, 100)

ax.set_xscale('log')
plt.savefig(fig_savepath + 'DiversityIndex3-SnowDepth.pdf', bbox_inches = 'tight')

Just Upland Data

In [ ]:
#Real data and Indicator 1 plot
fig, [ax2, ax] = plt.subplots(1, 2, figsize = (7, 3.5), 
                              layout = 'tight', 
                              sharex = True, 
                              sharey = True)

sns.regplot(data = diversityThree[diversityThree.zones == 'Upland'], x = 'PiInd', y = 'maxDepth',
            logx = True,
            ax = ax, color = pal[4])

#Do regression and add to plot
resultsUp = smf.ols('maxDepth ~ logPiInd', data = diversityThree[diversityThree.zones == 'Upland']).fit()
ax.text(18, 38, r'$ p $ = ' + str(round_sig(resultsUp.pvalues.logPiInd, 3)))
ax.text(18, 32, r'$ R_{adj}^2 $ = ' + str(round_sig(resultsUp.rsquared_adj, 3)))

ax.set_xlabel('Coniferous Canopy Class Index')
ax.set_ylabel('Snow Depth [cm]')
ax.set_xlim(np.nanmin(diversityThree.PiInd), np.nanmax(diversityThree.PiInd))

ax.set_xscale('log')
ax.text(0.6, 126, 'Upland Data', color = 'silver')

sns.regplot(data = diversityThree, x = 'PiInd', y = 'maxDepth',
            logx = True,
            ax = ax2, color = pal[4])

#Do regression and add to plot
results = smf.ols('maxDepth ~ logPiInd', data = diversityThree).fit()
ax2.text(18, 38, r'$ p $ = ' + str(round_sig(results.pvalues.logPiInd, 3)))
ax2.text(18, 32, r'$ R_{adj}^2 $ = ' + str(round_sig(results.rsquared_adj, 3)))

ax2.set_xlabel('Coniferous Canopy Class Index')
ax2.set_ylabel('Snow Depth [cm]')
ax2.set_xlim(np.nanmin(diversityThree.PiInd), np.nanmax(diversityThree.PiInd))

ax2.set_xscale('log')
ax2.text(0.6, 126, 'All Data', color = 'silver')

plt.savefig(fig_savepath + 'DiversityIndex3-SnowDepth-upland.pdf', bbox_inches = 'tight')

In [ ]:
resultsUp.params

#### Canopy Class Indicator #3 Melt Rates

In [ ]:
#Subset and merge with snow data
diversityThreeMelt = pd.merge(meltData[['stakes', 'melt_meas', 'zones', 'watershed', 'aspect', 'slope', 'lai5ring', 'nTrees', 'pCon']], fs[['CCCon', 'pCo', 'SITE']], left_on = 'stakes', right_on = 'SITE', how = 'inner')
diversityThreeMelt = diversityThreeMelt.rename(columns = {diversityThreeMelt.columns[-3] : 'CCCon',
                                            diversityThreeMelt.columns[-2] : 'pCo', 
                                            diversityThreeMelt.columns[-1] : 'SITE'})
diversityThreeMelt.head()

In [ ]:
#calculate non-dimensional variable
diversityThreeMelt['PiInd'] = diversityThreeMelt.pCon / (diversityThreeMelt.lai5ring*diversityThreeMelt.CCCon)
diversityThreeMelt['logPiInd'] = np.log(diversityThreeMelt['PiInd'])

#Change infs to nans
diversityThreeMelt = diversityThreeMelt.replace([np.inf, -np.inf], np.nan)
diversityThreeMelt.head()

In [ ]:
#Real data and Indicator 1 plot
fig, [ax2, ax] = plt.subplots(1, 2, figsize = (7, 3.5), 
                              layout = 'tight', 
                              sharex = True, 
                              sharey = True)

sns.regplot(data = diversityThreeMelt[diversityThreeMelt.zones == 'Upland'], x = 'PiInd', y = 'melt_meas',
            logx = True,
            ax = ax, color = pal[4])

#Do regression and add to plot
results = smf.ols('melt_meas ~ logPiInd', data = diversityThreeMelt[diversityThreeMelt.zones == 'Upland']).fit()
ax.text(15, -15, r'$ p $ = ' + str(round_sig(results.pvalues.logPiInd, 3)))
ax.text(15, -22, r'$ R_{adj}^2 $ = ' + str(round_sig(results.rsquared_adj, 3)))

ax.set_xlabel('Coniferous Canopy Class Index')
ax.set_ylabel('Melt Season Length [days]')
ax.set_xlim(np.nanmin(diversityThreeMelt.PiInd), np.nanmax(diversityThreeMelt.PiInd))

ax.set_xscale('log')
ax.text(0.6, 60, 'Upland Data', color = 'silver')

sns.regplot(data = diversityThreeMelt, x = 'PiInd', y = 'melt_meas',
            logx = True,
            ax = ax2, color = pal[4])

#Do regression and add to plot
results = smf.ols('melt_meas ~ logPiInd', data = diversityThreeMelt).fit()
ax2.text(15, -15, r'$ p $ = ' + str(round_sig(results.pvalues.logPiInd, 3)))
ax2.text(15, -22, r'$ R_{adj}^2 $ = ' + str(round_sig(results.rsquared_adj, 3)))

ax2.set_xlabel('Coniferous Canopy Class Index')
ax2.set_ylabel('Melt Season Length [days]')
ax2.set_xlim(np.nanmin(diversityThreeMelt.PiInd), np.nanmax(diversityThreeMelt.PiInd))
ax2.set_ylim(-25, 70)

ax2.set_xscale('log')
ax2.text(0.6, 60, 'All Data', color = 'silver')

plt.savefig(fig_savepath + 'DiversityIndex3-Melt-upland.pdf', bbox_inches = 'tight')

#### Canopy Class Indicator #3 Simulated

In [ ]:
#Cycle through a select range of trees
for ntr in range(1, 11):
    #Numerical Representation of Metric
    Sup = []
    Int = []
    Co = []
    Dom = []
    SupCon = []
    IntCon = []
    CoCon = []
    DomCon = []

    #Generate fake data for a site
    nTrees = ntr

    for nSup in np.arange(0, nTrees + 1):
        for nInt in np.arange(0, nTrees + 1 - nSup):
            for nCo in np.arange(0, nTrees + 1 - nSup - nInt):
                nDom = nTrees - nSup - nInt - nCo

                for SupConif in np.arange(0, nSup + 1):
                    for IntConif in np.arange(0, nInt + 1):
                        for CoConif in np.arange(0, nCo + 1):
                            for DomConif in np.arange(0, nDom + 1):

                                #Save data
                                Sup.append(nSup)
                                Int.append(nInt)
                                Co.append(nCo)
                                Dom.append(nDom)

                                #Save data
                                SupCon.append(SupConif)
                                IntCon.append(IntConif)
                                CoCon.append(CoConif)
                                DomCon.append(DomConif)

    #Append into dataframe
    IndicatorData = pd.DataFrame({'Sup' : Sup, 
                            'Int' : Int, 
                            'Co' : Co, 
                            'Dom' : Dom,
                            'SupConif' : SupCon, 
                            'IntConif' : IntCon, 
                            'CoConif' : CoCon, 
                            'DomConif' : DomCon})

    #Compute Variables
    lai_test = np.nanmean(peakData.lai5ring)
    IndicatorData['CCCon'] = np.where(np.isnan((IndicatorData['CoConif']/IndicatorData['Co'])), 0, IndicatorData['CoConif']/IndicatorData['Co'])
    IndicatorData['pCon'] = (IndicatorData['CoConif'] + IndicatorData['DomConif'] + IndicatorData['IntConif'] + IndicatorData['SupConif'])/(IndicatorData['Co'] + IndicatorData['Dom'] + IndicatorData['Int'] + IndicatorData['Sup'])
    IndicatorData['pCo'] = IndicatorData['Co']/(IndicatorData['Co'] + IndicatorData['Dom'] + IndicatorData['Int'] + IndicatorData['Sup'])
    IndicatorData['PiInd'] = IndicatorData.pCon / (lai_test*IndicatorData.CCCon)

    #Change infs to nans
    IndicatorData = IndicatorData.replace([np.inf, -np.inf], np.nan)

    #Plot
    temp = IndicatorData[(IndicatorData.pCo > 0)]
    sns.relplot(data = temp, x = jitter(temp['pCon'], 0.1), y = jitter(temp['CCCon'], 0.1),
                hue = 'PiInd', palette = 'Greens', 
                col = 'pCo', col_wrap = 5, 
                height = 3, aspect = 1)

    plt.savefig(fig_savepath + 'simulationfigs/IndicatorModel_nTrees_' + str(ntr) + '.pdf', 
                bbox_inches = 'tight')


In [ ]:
temp.head()

In [ ]:
#simulations with uniform tree numbers (n = 10) and lai set to average of MEF sites
fig, axs = plt.subplots(2, 5, figsize = (11, 5), 
                        sharex = True, 
                        sharey = True, 
                        layout = 'tight')

co = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
for i in range(0, 10):
    temp2 = temp[temp.pCo == co[i]]

    #Index Values
    yVal = temp2['pCon']/(temp2['CCCon'])
    
    #Model Snow Depth values
    temp2['SD'] = resultsUp.params.Intercept + resultsUp.params.logPiInd*np.log(temp2.PiInd)
    c = sns.scatterplot(data = temp2, x = jitter(yVal, 0.1), y = jitter(temp2['SD'], 0.1),
                color = pal[4], ax = axs.ravel()[i],
                linewidth = 0, alpha = 0.3, label = 'Snow Depth')
    d = sns.regplot(data = temp2, x = jitter(yVal, 0.1), y = jitter(temp2['SD'], 0.1),
                color = pal[4], ax = axs.ravel()[i],
                lowess = True, scatter = False)
    axs.ravel()[i].set_ylim(30, 70)

    #remove duplicate secondary y axes
    axs.ravel()[i].set_ylabel('Modelled Snow Depth [cm]')
        
    axs.ravel()[i].set_title('pCo = ' + str(co[i]), fontsize='small', loc='left')


plt.savefig(fig_savepath + 'DiversityIndex3-Simulations.pdf')


In [ ]:
#simulations with uniform tree numbers (n = 10), pCo at 0.5, and a range of LAI
fig, axs = plt.subplots(1, 5, figsize = (11, 3), 
                        sharex = True, 
                        sharey = True, 
                        layout = 'tight')

co = 0.5
temp2 = temp[temp.pCo == co]
lai = np.linspace(start = np.nanmin(peakData.lai5ring), stop = np.nanmax(peakData.lai5ring), num = 5)

for i in range(0, 5):
    temp2.PiInd = temp2.pCon / (lai[i]*temp2.CCCon)
    yVal = temp2.pCon / (temp2.CCCon)

    #Index Values
    a = sns.scatterplot(data = temp2, x = jitter(yVal, 0.1), y = jitter(temp2['PiInd'], 0.1),
                color = pal[2], ax = axs.ravel()[i], 
                linewidth = 0, alpha = 0.3, label = 'CCC Index')
    b = sns.regplot(data = temp2, x = jitter(yVal, 0.1), y = jitter(temp2['PiInd'], 0.1),
                color = pal[2], ax = axs.ravel()[i], 
                lowess = True, scatter = False)
    
    #Model Snow Depth values
    temp2['SD'] = resultsUp.params.Intercept + resultsUp.params.logPiInd*np.log(temp2.PiInd)
    ax2 = axs.ravel()[i].twinx()
    c = sns.scatterplot(data = temp2, x = jitter(yVal, 0.1), y = jitter(temp2['SD'], 0.1),
                color = pal[4], ax = ax2,
                linewidth = 0, alpha = 0.3, label = 'Snow Depth')
    d = sns.regplot(data = temp2, x = jitter(yVal, 0.1), y = jitter(temp2['SD'], 0.1),
                color = pal[4], ax = ax2,
                lowess = True, scatter = False)
    ax2.set_ylim(30, 100)

    #remove duplicate secondary y axes
    if(i%5 != 4):
        ax2.set_axis_off()
    else:
        ax2.set_ylabel('Modelled Snow Depth [cm]')
        
    #add labels 
    if(i != 0):
        axs.ravel()[i].get_legend().remove()
        ax2.get_legend().remove()
    
    axs.ravel()[i].set_yscale('log')
    axs.ravel()[i].set_xlabel(r'$pCCC$', fontsize = 'large')
    axs.ravel()[i].set_ylabel('CCC Index')
    axs.ravel()[i].set_title('LAI = ' + str(round(lai[i], 3)), fontsize='small', loc='left')


plt.savefig(fig_savepath + 'DiversityIndex3-Simulations-varyLAI.pdf')

In [ ]:
IndicatorData['pCCC'] = IndicatorData['pCon']/(IndicatorData['CCCon'])
IndicatorData = IndicatorData.replace([np.inf, -np.inf], np.nan)

fig, ax = plt.subplots(1, 1, figsize = (4, 4))

sns.scatterplot(x = jitter(IndicatorData['Co'], 0.1), y = jitter(IndicatorData['CoConif'], 0.1),
                 hue = IndicatorData['pCCC'], palette = 'Greens', 
                 ax = ax)

ax.set_xlabel('Number of Co-dominant Trees')
ax.set_ylabel('Number of Coniferous Co-dominant Trees')
plt.legend(title = r'$pCCC}$')

plt.savefig(fig_savepath + "DiversityIndex3-Simulations-CCCfrac.pdf")

### Canopy Class Inidicator #4

In [ ]:
fig, [ax1, ax2] = plt.subplots(1, 2, figsize = (6, 3), 
                               layout = 'tight', 
                               sharey = True)

#Proportion of co-dominant trees
sns.regplot(x = dd[(dd.SYear == 2023)].pCoDom, y = dd[(dd.SYear == 2023)].maxDepth, order = 2, ax = ax1, color = pal[4])
sns.regplot(x = dd[(dd.SYear == 2024)].pCoDom, y = dd[(dd.SYear == 2024)].maxDepth, order = 2, ax = ax1, color = pal[2])
#Do regression and add to plot
results3_high = smf.ols('maxDepth ~ pCoDom2', data = dd[(dd.SYear == 2023)]).fit()
results3_low = smf.ols('maxDepth ~ pCoDom2', data = dd[(dd.SYear == 2024)]).fit()
ax1.text(0.1, 92, 
        r'$ p $ = ' + str(round_sig(results3_high.pvalues.pCoDom2, 2)) + r', $ R_{adj}^2 $ = ' + str(round_sig(results3_high.rsquared_adj, 2)), 
        color = pal[4])
ax1.text(0.1, 84,
        r'$ p $ = ' + str(round_sig(results3_low.pvalues.pCoDom2, 2)) + r', $ R_{adj}^2 $ = ' + str(round_sig(results3_low.rsquared_adj, 2)), 
        color = pal[2])

ax1.set_xlabel('Proportion of Co-Dominant Trees')
ax1.set_ylabel(' ')
ax1.set_xlim(min(dd.pCoDom), max(dd.pCoDom))
ax1.set_ylim(10, 100)

#Diversity Index4
dd['CoH_Index'] = (np.log(dd.pCon + 1))*(dd.pCoDom2)
dd = dd.replace([np.inf, -np.inf], np.nan)
sns.regplot(x = dd[(dd.SYear == 2023)].CoH_Index, y = dd[(dd.SYear == 2023)].maxDepth, order = 2, ax = ax2, color = pal[4])
sns.regplot(x = dd[(dd.SYear == 2024)].CoH_Index, y = dd[(dd.SYear == 2024)].maxDepth, order = 2, ax = ax2, color = pal[2])
#Do regression and add to plot
results2_high = smf.ols('maxDepth ~ CoH_Index', data = dd[(dd.SYear == 2023)]).fit()
results2_low = smf.ols('maxDepth ~ CoH_Index', data = dd[(dd.SYear == 2024)]).fit()
ax2.text(0.1, 92, 
        r'$ p $ = ' + str(round_sig(results2_high.pvalues.CoH_Index, 2)) + r', $ R_{adj}^2 $ = ' + str(round_sig(results2_high.rsquared_adj, 2)), 
        color = pal[4])
ax2.text(0.1, 84,
        r'$ p $ = ' + str(round_sig(results2_low.pvalues.CoH_Index, 2)) + r', $ R_{adj}^2 $ = ' + str(round_sig(results2_low.rsquared_adj, 2)), 
        color = pal[2])
#ax2.set_xscale('log')
ax2.set_xlabel('Diversity Index 4')
ax2.set_ylabel(' ')
ax2.set_xlim(np.nanmin(dd.CoH_Index), np.nanmax(dd.CoH_Index))
ax2.set_ylim(10, 100)

In [ ]:
diversityFour = pd.merge(peakData[['stakes', 'maxDepth', 'zones', 'watershed', 'aspect', 'slope', 'lai5ring', 'nTrees', 'pCon', 'SYear']], fs[['CCCon', 'CCCon_ntrees', 'pCo', 'SITE']], left_on = 'stakes', right_on = 'SITE', how = 'inner')
diversityFour = diversityFour.rename(columns = {diversityFour.columns[-4] : 'CCCon',
                                            diversityFour.columns[-3] : 'CCCon_tree',
                                            diversityFour.columns[-2] : 'pCo', 
                                            diversityFour.columns[-1] : 'SITE'})
diversityFour.head()

In [ ]:
#calculate non-dimensional variable
diversityFour['Ind'] = (diversityFour.lai5ring*(np.sqrt(diversityFour.pCon))*(diversityFour.pCo*diversityFour.pCo))
diversityFour['IndLog'] = (diversityFour.lai5ring*(np.log(diversityFour.pCon + 1))*(diversityFour.pCo*diversityFour.pCo))

In [ ]:
diversityFour.head()

#### Simulation

In [ ]:
#Numerical Representation of Metric
Sup = []
Int = []
Co = []
Dom = []
SupCon = []
IntCon = []
CoCon = []
DomCon = []
H = []
S = []
divFour = []

#Generate fake data for a site
nTrees = 10

for nSup in np.arange(0, nTrees + 1):
    for nInt in np.arange(0, nTrees + 1 - nSup):
        for nCo in np.arange(0, nTrees + 1 - nSup - nInt):
            nDom = nTrees - nSup - nInt - nCo

            for SupConif in np.arange(0, nSup + 1):
                for IntConif in np.arange(0, nInt + 1):
                    for CoConif in np.arange(0, nCo + 1):
                        for DomConif in np.arange(0, nDom + 1):

                            totConif = SupConif + IntConif + CoConif + DomConif

                            #Save data
                            Sup.append(nSup)
                            Int.append(nInt)
                            Co.append(nCo)
                            Dom.append(nDom)

                            #Save data
                            SupCon.append(SupConif)
                            IntCon.append(IntConif)
                            CoCon.append(CoConif)
                            DomCon.append(DomConif)

                            #Calculate Fourth Diversity Index (Square Root)
                            divFour.append((np.nanmean(diversityFour.lai5ring)*(np.sqrt(totConif/nTrees))*((nCo/nTrees)*(nCo/nTrees))))

In [ ]:
#Append into dataframe
divFour_modelled = pd.DataFrame({'Sup' : Sup, 
                         'Int' : Int, 
                         'Co' : Co, 
                         'Dom' : Dom,
                         'SupConif' : SupCon, 
                         'IntConif' : IntCon, 
                         'CoConif' : CoCon, 
                         'DomConif' : DomCon, 
                         'divFour' : divFour})

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (4, 4))

sns.scatterplot(x = jitter(divFour_modelled['Co'], 0.1), y = jitter(divFour_modelled['CoConif'], 0.1),
                 hue = divFour_modelled['divFour'], palette = 'Greens', 
                 ax = ax)

ax.set_xlabel('Number of Co-dominant Trees')
ax.set_ylabel('Number of Coniferous Co-dominant Trees')
plt.legend(title = r'$\Pi_2}$')

plt.savefig(fig_savepath + "DiversityIndex4-Simulations-SqrtPI.pdf")

### Canopy Class Index #5

In [ ]:
dd.head()

## Index Summary


In [ ]:
fig, [ax, ax2, ax3] = plt.subplots(1, 3, figsize = (9, 3), 
                                             sharey = True, 
                                             layout = 'tight')

#Diversity Three (Pi Theorem Index)
sns.regplot(data = diversityThree[diversityThree.SYear == 2023], x = 'PiInd', y = 'maxDepth',
            logx = True,
            ax = ax, color = pal[4])
sns.regplot(data = diversityThree[diversityThree.SYear == 2024], x = 'PiInd', y = 'maxDepth',
            logx = True,
            ax = ax, color = pal[2])
res1 = smf.ols('maxDepth ~ logPiInd', data = diversityThree[(diversityThree.SYear == 2023)]).fit()
res2 = smf.ols('maxDepth ~ logPiInd', data = diversityThree[(diversityThree.SYear == 2024)]).fit()
ax.text(-0.5, 92,
        r'$ p $ = ' + str(round_sig(res1.pvalues.logPiInd, 2)) + r', $ R_{adj}^2 $ = ' + str(round_sig(res1.rsquared_adj, 2)), 
        color = pal[4])
ax.text(-0.5, 84, r'$ p $ = ' + str(round_sig(res2.pvalues.logPiInd, 2)) + r', $ R_{adj}^2 $ = ' + str(round_sig(res2.rsquared_adj, 2)), 
        color = pal[2])
ax.set_xlim(np.nanmin(diversityThree.logPiInd), np.nanmax(diversityThree.logPiInd))
ax.text(-0.5, 76, r'$ n $ = ' + str(len(res1.fittedvalues) + len(res2.fittedvalues)))
ax.set_xlabel('Index')
ax.set_ylabel('Snow Depth [cm]')
ax.set_title('Pi Theorem Index (log)', fontsize='small', loc='left')

#Diversity Four Square Root
sns.regplot(x = diversityFour[(diversityFour.SYear == 2023)].Ind, y = diversityFour[(diversityFour.SYear == 2023)].maxDepth, ax = ax2, color = pal[4])
sns.regplot(x = diversityFour[(diversityFour.SYear == 2024)].Ind, y = diversityFour[(diversityFour.SYear == 2024)].maxDepth, ax = ax2, color = pal[2])
#Do regression and add to plot
results_high = smf.ols('maxDepth ~ Ind', data = diversityFour[(diversityFour.SYear == 2023)]).fit()
results_low = smf.ols('maxDepth ~ Ind', data = diversityFour[(diversityFour.SYear == 2024)]).fit()
ax2.text(0.05, 92, 
        r'$ p $ = ' + str(round_sig(results_high.pvalues.Ind, 2)) + r', $ R_{adj}^2 $ = ' + str(round_sig(results_high.rsquared_adj, 2)), 
        color = pal[4])
ax2.text(0.05, 84,
        r'$ p $ = ' + str(round_sig(results_low.pvalues.Ind, 2)) + r', $ R_{adj}^2 $ = ' + str(round_sig(results_low.rsquared_adj, 2)), 
        color = pal[2])
ax2.text(0.05, 76, r'$ n $ = ' + str(len(results_high.fittedvalues) + len(results_low.fittedvalues)))
ax2.set_xlim(np.nanmin(diversityFour.Ind), np.nanmax(diversityFour.Ind))
ax2.set_ylabel(' ')
ax2.set_xlabel('Index')
ax2.set_title('Regression Index - Square root', fontsize='small', loc='left')

#Diversity Four Logarithm
sns.regplot(x = diversityFour[(diversityFour.SYear == 2023)].IndLog, y = diversityFour[(diversityFour.SYear == 2023)].maxDepth, ax = ax3, color = pal[4])
sns.regplot(x = diversityFour[(diversityFour.SYear == 2024)].IndLog, y = diversityFour[(diversityFour.SYear == 2024)].maxDepth, ax = ax3, color = pal[2])
#Do regression and add to plot
results2_high = smf.ols('maxDepth ~ IndLog', data = diversityFour[(diversityFour.SYear == 2023)]).fit()
results2_low = smf.ols('maxDepth ~ IndLog', data = diversityFour[(diversityFour.SYear == 2024)]).fit()
ax3.text(0.05, 92, 
        r'$ p $ = ' + str(round_sig(results2_high.pvalues.IndLog, 2)) + r', $ R_{adj}^2 $ = ' + str(round_sig(results2_high.rsquared_adj, 2)), 
        color = pal[4])
ax3.text(0.05, 84,
        r'$ p $ = ' + str(round_sig(results2_low.pvalues.IndLog, 2)) + r', $ R_{adj}^2 $ = ' + str(round_sig(results2_low.rsquared_adj, 2)), 
        color = pal[2])
ax3.text(0.05, 76, r'$ n $ = ' + str(len(results2_high.fittedvalues) + len(results2_low.fittedvalues)))
ax3.set_xlim(np.nanmin(diversityFour.IndLog), np.nanmax(diversityFour.IndLog))
ax3.set_ylabel(' ')
ax3.set_xlabel('Index')
ax3.set_title('Regression Index - Log', fontsize='small', loc='left')

ax.set_ylim(10, 100)

plt.savefig(fig_savepath + 'index_summary.pdf')


What this says to me is that snow depth is more dependent on the interaction of LAI and tree structure in dense coniferous canopies -- whereas LAI is a perfectly fine predictor in deciduous canopies.

## Do it all with peak SWE

With all of this, how do we know what we are seeing isn't just the effects of shading? 

In [ ]:
swe = pd.read_csv(import_path + '2023SWE_data.csv')
swe.head()

In [ ]:
peakDataSWE = swe.groupby(['SITE'])['SWE_cm'].max().reset_index()
peakDataSWE = peakDataSWE.rename(columns = {'depths' : 'maxDepth'})

lai_sub = lai_df[['Stake_ID', 'OLS Prediction Ring 5']]
peakDataSWE = pd.merge(peakDataSWE, lai_sub, left_on = 'SITE', right_on = 'Stake_ID')
peakDataSWE = peakDataSWE.drop(columns = 'Stake_ID')
peakDataSWE = peakDataSWE.rename(columns = {'OLS Prediction Ring 5' : 'lai5ring'})

veg_sub = forestInv[['Stake_ID', 'DIST_M', 'DBH_CM', 'DIST_M_SD', 'DBH_CM_SD', 'n', 'basalArea_m2', 'Co', 'Dom', 'Int', 'Sup', 'prop_Coniferous']]
peakDataSWE = pd.merge(peakDataSWE, veg_sub, left_on = 'SITE', right_on = 'Stake_ID')
peakDataSWE = peakDataSWE.drop(columns = 'Stake_ID')
peakDataSWE = peakDataSWE.rename(columns = {'DIST_M' : 'avgDist_m',
                                    'DBH_CM' : 'avgDBH_cm',
                                    'DIST_M_SD' : 'sdDist_m',
                                    'DBH_CM_SD' : 'sdDBH_cm',
                                    'n' : 'nTrees',
                                    'Co' : 'nCo',
                                    'Dom' : 'nDom',
                                    'Int' : 'nInt',
                                    'Sup' : 'nSup',
                                    'prop_Coniferous' : 'pCon'})

In [ ]:
peakDataSWE.head()

In [ ]:
'''Set PCA Data'''
#Switch this depending on average/maximum snow depth considerations
pca_data = peakDataSWE

#remove strings
pca_data_forplot = pca_data.drop(columns = ['SITE'])

#remove depths 
pca_data = pca_data_forplot.drop(columns = 'SWE_cm')

In [ ]:
#Split data
X_train_SWE, X_test_SWE, y_train_SWE, y_test_SWE = train_test_split(pca_data, pca_data_forplot.SWE_cm,
                                                                        test_size = 0.5, random_state = 48492)

#Fit random forest regression with 100 trees
rfSWE = RandomForestRegressor(n_estimators = 100)
rfSWE.fit(X_train_SWE, y_train_SWE)

In [ ]:
sorted_idxSWE = rfSWE.feature_importances_.argsort()

fig,[ax, ax2] = plt.subplots(2, 1, figsize = (6, 5), 
                              sharex = True, 
                              tight_layout = True)

#2023
#Without permutation
ax.barh(pca_data.columns[sorted_idxSWE], rfSWE.feature_importances_[sorted_idxSWE], 
        color = pal[4])
#ax.set_yticklabels(['Aspect', 'Northing', '# Sup', '# Int', 'Mean Dist', '# Trees', 'Easting', 'Slope', 'Mean DBH', '# Dom', 'LAI', 'BA', '% Con', '# Co'])

#With permutation
perm_importanceSWE = permutation_importance(rfSWE, X_test_SWE, y_test_SWE)

sorted_idxSWE = perm_importanceSWE.importances_mean.argsort()
ax2.barh(pca_data.columns[sorted_idxSWE], perm_importanceSWE.importances_mean[sorted_idxSWE], 
         color = pal[4])
#ax2.set_yticklabels(['# Dom', '# Sup', '# Int', 'Easting', 'Aspect', 'Mean Dist', 'BA', '# Trees', 'Slope', 'Northing', 'Mean DBH', 'LAI', '% Con', '# Co'])


In [ ]:
explainerSWE = shap.TreeExplainer(rfSWE)
shap_valuesSWE = explainerSWE.shap_values(X_test_SWE)


shap.summary_plot(shap_valuesSWE, X_test_SWE,
                cmap = 'GnBu', 
                plot_size = (5, 5), 
                color_bar_label = 'SWE Value', 
                use_log_scale = True, 
                show = False)

## Soil Moisture and Drydown 

In [ ]:
#Import Drydown data
drydown = pd.read_csv(import_path + 'soilMoisture_Drawdown.csv')
drydown.head()

In [ ]:
#Merge with veg data and the regression index data
drydown_veg = pd.merge(drydown, diversityFour, right_on = 'SITE', left_on = 'Site', how = 'inner')
drydown_veg.head()

In [ ]:
fig, [ax, ax2, ax3] = plt.subplots(1, 3, figsize = (11, 4), 
                            sharex = True, 
                            layout = 'tight')

sns.violinplot(data = drydown_veg, x = 'Bin', y = 'BeginDOY',
            color = 'white', ax = ax)
sns.stripplot(data = drydown_veg, x = 'Bin', y = 'BeginDOY',
            color = pal[3], ax = ax,
            zorder = 4)
ax.set_xlabel('Canopy Classification')
ax.set_ylabel('Beginning of Summer Drawdown, Day of Year')

sns.violinplot(data = drydown_veg, x = 'Bin', y = 'Drawdown',
            color = 'white', ax = ax2)
sns.stripplot(data = drydown_veg, x = 'Bin', y = 'Drawdown',
            color = pal[3], ax = ax2,
            zorder = 4)
ax2.set_xlabel('Canopy Classification')
ax2.set_ylabel('Soil Moisture Drawdown [cm3/cm3]')

sns.violinplot(data = drydown_veg, x = 'Bin', y = 'EndAmt',
            color = 'white', ax = ax3)
sns.stripplot(data = drydown_veg, x = 'Bin', y = 'EndAmt',
            color = pal[3], ax = ax3,
            zorder = 4)
ax3.set_xlabel('Canopy Classification')
ax3.set_ylabel('End of Summer Soil Moisture [cm3/cm3]')

plt.savefig(fig_savepath + 'drawdown_violinplots.pdf')